# Linear Combination of Fourier Basis

## 1D Discrete Fourier Transform

For a discrete signal $x[n]$ of length $N$:

$$
x[n] = \frac{1}{\sqrt{N}} \sum_{k=0}^{N-1} S[k] \, e^{\, j \, 2\pi \frac{kn}{N}}, 
\quad n = 0, 1, \dots, N-1
$$

- $e^{j 2\pi kn/N}$ are the **Fourier basis functions** (complex sinusoids).  
- $S[k]$ are the **Fourier coefficients** (weights).  
- The signal $x[n]$ is a **linear combination** of these basis functions.  

In matrix-vector form:

$$
\mathbf{x} = \boldsymbol{\Psi} \, \mathbf{S}
$$

- $\mathbf{x}$ is the $N \times 1$ signal vector.  
- $\boldsymbol{\Psi}$ is the $N \times N$ Fourier basis matrix.  
- $\mathbf{S}$ is the $N \times 1$ coefficient vector.  

---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objects as go

# =========================================================
# Generate synthetic 1D signal
# =========================================================
def generate_signal(n):
    t = np.linspace(0, 1, n, endpoint=False)
    # Mix of sine waves
    signal = (
        0.7 * np.sin(2 * np.pi * 5 * t)
        + 0.5 * np.sin(2 * np.pi * 12 * t)
        + 0.3 * np.sin(2 * np.pi * 20 * t)
    )
    return signal, t

# =========================================================
# Interactive function
# =========================================================
def show_all(percent=10, n=64):
    X, t = generate_signal(n)

    # Fourier basis (DFT matrix)
    Psi = np.fft.fft(np.eye(n)) / np.sqrt(n)

    # =====================================================
    # Fourier sparsity + reconstruction (matplotlib row 1)
    # =====================================================
    F = np.fft.fft(X)
    Fshift = np.fft.fftshift(F)
    F_abs = np.abs(Fshift)

    threshold = np.percentile(F_abs, 100 - percent) if percent > 0 else np.inf
    Fshift_sparse = np.where(F_abs >= threshold, Fshift, 0)

    # Reconstruction
    F_ishift = np.fft.ifftshift(Fshift_sparse)
    X_reconstructed = np.fft.ifft(F_ishift).real

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    # Original signal
    axes[0].plot(t, X, color="black")
    axes[0].set_title("Original Signal", fontname="Times New Roman")

    # Fourier coefficients
    axes[1].stem(np.linspace(-n//2, n//2-1, n), np.abs(Fshift), basefmt=" ")
    axes[1].set_title(f"Fourier Spectrum", fontname="Times New Roman")

    # Reconstructed signal
    axes[2].plot(t, X_reconstructed, color="orange")
    axes[2].set_title(f"Reconstructed ({percent}% coeffs)", fontname="Times New Roman")

    plt.tight_layout()
    plt.show()

    # =====================================================
    # Plotly visualization of X, Ψ, S (row 2)
    # =====================================================
    # Fourier coefficients (vector form)
    S = (np.linalg.inv(Psi) @ X).real

    # Sparsity mask
    S_abs = np.abs(S)
    threshold_S = np.percentile(S_abs, 100 - percent) if percent > 0 else np.inf
    S_sparse_mask = (S_abs >= threshold_S).astype(int)

    fig = go.Figure()

    # Fixed display size
    display_size = 256
    vector_width = 36
    block_gap = 120
    base_x = 0

    # X vector
    X_disp = resize(
        X.reshape(n,1),
        (display_size,1),
        anti_aliasing=False,
        order=0   # preserves discreteness
    )

    fig.add_trace(go.Heatmap(
        z=X_disp.repeat(vector_width, axis=1),
        colorscale="Gray",
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="X"
    ))
    fig.add_annotation(
        x=base_x + vector_width/2, y=-12, text="X (n×1)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )
    base_x += vector_width + block_gap

    # Ψ matrix (resized for display)
    Psi_disp = resize(
        Psi.real,
        (display_size, display_size),
        anti_aliasing=False,
        order=0   # nearest-neighbor → preserves discreteness
        )

    fig.add_trace(go.Heatmap(
        z=Psi_disp,
        colorscale="RdBu",
        zmid=0,
        showscale=False,
        x=list(range(base_x, base_x + display_size)),
        y=list(range(display_size)),
        name="Psi"
    ))
    fig.add_annotation(
        x=base_x + display_size/2, y=-12, text="Ψ (n×n)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )
    base_x += display_size + block_gap

    # S vector (mask, resized for display)
    S_disp = resize(
        S_sparse_mask.reshape(n,1),
        (display_size,1),
        anti_aliasing=False,
        order=0
    )
    fig.add_trace(go.Heatmap(
        z=S_disp.repeat(vector_width, axis=1),
        colorscale=[[0, "black"], [1, "orange"]],
        zmin=0, zmax=1,
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="S"
    ))
    fig.add_annotation(
        x=base_x + vector_width/2, y=-12,
        text=f"S support (top {percent}%)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )

    # Layout
    fig.update_layout(
        font=dict(family="Times New Roman"),
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, scaleanchor="x"),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=40, b=60),
        height=500,
        width=1000
    )

    fig.show()

# =========================================================
# Interactive sliders
# =========================================================
interact(
    show_all,
    percent=widgets.IntSlider(min=0, max=100, step=1, value=10, description="Coeffs %"),
    n=widgets.IntSlider(min=16, max=256, step=16, value=64, description="Signal length")
)


interactive(children=(IntSlider(value=10, description='Coeffs %'), IntSlider(value=64, description='Signal len…

<function __main__.show_all(percent=10, n=64)>

In [2]:
import plotly.io as pio

# =========================================================
# Save top row (matplotlib) as PDF
# =========================================================
def save_top_row_1d(percent=10, n=64, filename="top_row_1d.pdf"):
    X, t = generate_signal(n)

    # Fourier
    F = np.fft.fft(X)
    Fshift = np.fft.fftshift(F)
    F_abs = np.abs(Fshift)

    threshold = np.percentile(F_abs, 100 - percent) if percent > 0 else np.inf
    Fshift_sparse = np.where(F_abs >= threshold, Fshift, 0)

    # Reconstruction
    F_ishift = np.fft.ifftshift(Fshift_sparse)
    X_reconstructed = np.fft.ifft(F_ishift).real

    fig, axes = plt.subplots(1, 3, figsize=(14, 4))

    # Original signal
    axes[0].plot(t, X, color="black")
    axes[0].set_title("Original Signal", fontname="Times New Roman")
    axes[0].axis("off")

    # Fourier spectrum
    axes[1].stem(np.linspace(-n//2, n//2-1, n), np.abs(Fshift), basefmt=" ")
    axes[1].set_title("Fourier Spectrum", fontname="Times New Roman")
    axes[1].axis("off")

    # Reconstructed signal
    axes[2].plot(t, X_reconstructed, color="orange")
    axes[2].set_title(f"Reconstructed ({percent}% coeffs)", fontname="Times New Roman")
    axes[2].axis("off")

    # Remove all margins
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)

    plt.savefig(filename, bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    print(f"✅ Saved top row (1D) to {filename}")


# =========================================================
# Save bottom row (plotly) as PDF
# =========================================================
def save_bottom_row_1d(percent=10, n=64, filename="bottom_row_1d.pdf"):
    X, t = generate_signal(n)

    # Fourier basis
    Psi = np.fft.fft(np.eye(n)) / np.sqrt(n)
    S = (np.linalg.inv(Psi) @ X).real

    # Sparsity mask
    S_abs = np.abs(S)
    threshold_S = np.percentile(S_abs, 100 - percent) if percent > 0 else np.inf
    S_sparse_mask = (S_abs >= threshold_S).astype(int)

    # Display parameters
    display_size = 256
    vector_width = 36
    block_gap = 120
    base_x = 0

    fig = go.Figure()

    # --- X vector (nearest-neighbor resize → discrete blocks)
    # --- X vector (quantized to discrete grayscale)
    X_norm = (X - X.min()) / (X.max() - X.min()) * 255
    X_int = X_norm.astype(int)
    X_disp = np.repeat(X_int.reshape(n, 1), display_size // n, axis=0)

    fig.add_trace(go.Heatmap(
        z=X_disp.repeat(vector_width, axis=1),
        colorscale="Gray",
        zmin=0, zmax=255,
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="X"
    ))

    fig.add_trace(go.Heatmap(
        z=X_disp.repeat(vector_width, axis=1),
        colorscale="Gray",
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="X"
    ))
    base_x += vector_width + block_gap

    # --- Psi matrix
    Psi_disp = resize(
        Psi.real,
        (display_size, display_size),
        anti_aliasing=False,
        order=0
    )
    fig.add_trace(go.Heatmap(
        z=Psi_disp,
        colorscale="RdBu",
        zmid=0,
        showscale=False,
        x=list(range(base_x, base_x + display_size)),
        y=list(range(display_size)),
        name="Psi"
    ))
    base_x += display_size + block_gap

    # --- S vector
    S_disp = resize(
        S_sparse_mask.reshape(n,1),
        (display_size,1),
        anti_aliasing=False,
        order=0
    )
    fig.add_trace(go.Heatmap(
        z=S_disp.repeat(vector_width, axis=1),
        colorscale=[[0, "black"], [1, "orange"]],
        zmin=0, zmax=1,
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="S"
    ))

    # Layout (no margins)
    fig.update_layout(
        font=dict(family="Times New Roman"),
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, scaleanchor="x"),
        plot_bgcolor="white",
        margin=dict(l=0, r=0, t=0, b=0),
        height=500,
        width=1000
    )

    # Export as PDF
    pio.write_image(fig, filename, format="pdf")
    print(f"✅ Saved bottom row (1D) to {filename}")



In [4]:
save_top_row_1d(percent=5, n=256, filename="CS-top1d.pdf")
save_bottom_row_1d(percent=5, n=256, filename="CS-bottom1d.pdf")


✅ Saved top row (1D) to CS-top1d.pdf


ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido




## 2D Discrete Fourier Transform (Images)

For an image $X[u,v]$ of size $N \times N$:

$$
X[u,v] = \frac{1}{N} \sum_{k=0}^{N-1} \sum_{\ell=0}^{N-1} 
S[k,\ell] \, e^{\, j \, 2\pi \left( \frac{ku}{N} + \frac{\ell v}{N} \right)}
$$

- The **basis functions** are 2D sinusoids:

$$
e^{\, j \, 2\pi \left( \frac{ku}{N} + \frac{\ell v}{N} \right)}
$$

- The **coefficients** $S[k,\ell]$ specify the contribution of each basis.  
- The original image is exactly a **linear combination** of these 2D basis patterns.  

---

✅ **Key Idea:**  
Every signal or image can be reconstructed as a weighted sum of Fourier basis functions.  
The Fourier coefficients are the weights, and the basis functions are the building blocks.


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage.color import rgb2gray
from skimage.transform import resize
import ipywidgets as widgets
from ipywidgets import interact
import plotly.graph_objects as go

# =========================================================
# Load and preprocess image
# =========================================================
img = mpimg.imread("portrait.jpg")

# convert to grayscale
if img.ndim == 3:
    img_gray = rgb2gray(img)
else:
    img_gray = img

# =========================================================
# Interactive function
# =========================================================
def show_all(percent=5, n_side=32):
    # Resize image for both reconstruction and matrix visualization
    img_resized = resize(img_gray, (n_side, n_side), anti_aliasing=True)
    X = img_resized.flatten()
    n = X.size

    # Fourier basis
    Psi = np.fft.fft(np.eye(n)) / np.sqrt(n)

    # =====================================================
    # Fourier sparsity + reconstruction (matplotlib row 1)
    # =====================================================
    F = np.fft.fft2(img_resized)
    Fshift = np.fft.fftshift(F)
    F_abs = np.abs(Fshift)

    threshold = np.percentile(F_abs, 100 - percent) if percent > 0 else np.inf
    Fshift_sparse = np.where(F_abs >= threshold, Fshift, 0)

    # Sparse spectrum
    magnitude_sparse = np.log1p(np.abs(Fshift_sparse))

    # Reconstruction
    F_ishift = np.fft.ifftshift(Fshift_sparse)
    img_reconstructed = np.fft.ifft2(F_ishift).real

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))  # fixed figure size

    # Original
    axes[0].imshow(img_resized, cmap="gray")
    axes[0].set_title("Original Image", fontname="Times New Roman")
    axes[0].axis("off")

    # Chosen coefficients
    axes[1].imshow(magnitude_sparse, cmap="magma")
    axes[1].set_title(f"Top {percent}% Fourier Coeffs", fontname="Times New Roman")
    axes[1].axis("off")

    # Reconstruction
    axes[2].imshow(np.clip(img_reconstructed, 0, 1), cmap="gray")
    axes[2].set_title(f"Reconstructed ({percent}% coeffs)", fontname="Times New Roman")
    axes[2].axis("off")

    plt.tight_layout()
    plt.show()

    # =====================================================
    # Plotly visualization of X, Ψ, S (row 2)
    # =====================================================
    # Fourier coefficients
    S = (np.linalg.inv(Psi) @ X).real

    # Sparsity mask (0 or 1)
    S_abs = np.abs(S)
    threshold_S = np.percentile(S_abs, 100 - percent) if percent > 0 else np.inf
    S_sparse_mask = (S_abs >= threshold_S).astype(int)

    fig = go.Figure()

    # Fixed display size (independent of n)
    display_size = 256   # controls readability
    vector_width = 36
    block_gap = 120
    base_x = 0

    # X vector (resized for display)
    X_disp = resize(X.reshape(n,1), (display_size,1), anti_aliasing=False)
    fig.add_trace(go.Heatmap(
        z=X_disp.repeat(vector_width, axis=1),
        colorscale="Gray",
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="X"
    ))
    fig.add_annotation(
        x=base_x + vector_width/2, y=-12, text="X (n×1)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )
    base_x += vector_width + block_gap

    # Ψ matrix (resized square for display)
    Psi_disp = resize(Psi.real, (display_size, display_size), anti_aliasing=False)
    fig.add_trace(go.Heatmap(
        z=Psi_disp,
        colorscale="RdBu",
        zmid=0,
        showscale=False,
        x=list(range(base_x, base_x + display_size)),
        y=list(range(display_size)),
        name="Psi"
    ))
    fig.add_annotation(
        x=base_x + display_size/2, y=-12, text="Ψ (n×n)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )
    base_x += display_size + block_gap

    # S vector (mask, nearest-neighbor resize to keep binary)
    S_disp = resize(
        S_sparse_mask.reshape(n,1),
        (display_size,1),
        anti_aliasing=False,
        order=0  # nearest-neighbor → keeps 0/1
    )

    fig.add_trace(go.Heatmap(
        z=S_disp.repeat(vector_width, axis=1),
        colorscale=[[0, "black"], [1, "orange"]],  # binary colormap
        zmin=0, zmax=1,
        showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size)),
        name="S"
    ))
    fig.add_annotation(
        x=base_x + vector_width/2, y=-12,
        text=f"S support (top {percent}%)",
        showarrow=False, font=dict(size=18, family="Times New Roman")
    )

    # Layout (fixed size, stable regardless of n)
    fig.update_layout(
        font=dict(family="Times New Roman"),
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, scaleanchor="x"),
        plot_bgcolor="white",
        margin=dict(l=20, r=20, t=40, b=60),
        height=600,
        width=1200
    )

    fig.show()

# =========================================================
# Interactive sliders
# =========================================================
interact(
    show_all,
    percent=widgets.IntSlider(min=0, max=100, step=1, value=20, description="Coeffs %"),
    n_side=widgets.IntSlider(min=8, max=64, step=2, value=46, description="Image size")
)


interactive(children=(IntSlider(value=20, description='Coeffs %'), IntSlider(value=46, description='Image size…

<function __main__.show_all(percent=5, n_side=32)>

In [ ]:
import matplotlib.pyplot as plt
import plotly.io as pio

def save_top_row(percent=5, n_side=32, filename="top_row.pdf"):
    img_resized = resize(img_gray, (n_side, n_side), anti_aliasing=True)
    F = np.fft.fft2(img_resized)
    Fshift = np.fft.fftshift(F)
    F_abs = np.abs(Fshift)

    threshold = np.percentile(F_abs, 100 - percent) if percent > 0 else np.inf
    Fshift_sparse = np.where(F_abs >= threshold, Fshift, 0)
    magnitude_sparse = np.log1p(np.abs(Fshift_sparse))

    F_ishift = np.fft.ifftshift(Fshift_sparse)
    img_reconstructed = np.fft.ifft2(F_ishift).real

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].imshow(img_resized, cmap="gray")
    axes[0].set_title("Original Image", fontname="Times New Roman")
    axes[0].axis("off")

    axes[1].imshow(magnitude_sparse, cmap="magma")
    axes[1].set_title(f"Top {percent}% Fourier Coeffs", fontname="Times New Roman")
    axes[1].axis("off")

    axes[2].imshow(np.clip(img_reconstructed, 0, 1), cmap="gray")
    axes[2].set_title(f"Reconstructed ({percent}% coeffs)", fontname="Times New Roman")
    axes[2].axis("off")

    # remove all subplot padding and margins
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)

    plt.savefig(filename, bbox_inches="tight", pad_inches=0)
    plt.close(fig)
    print(f"✅ Saved top row to {filename} (no white margins)")



def save_bottom_row(percent=5, n_side=32, filename="bottom_row.pdf"):
    img_resized = resize(img_gray, (n_side, n_side), anti_aliasing=True)
    X = img_resized.flatten()
    n = X.size

    Psi = np.fft.fft(np.eye(n)) / np.sqrt(n)
    S = (np.linalg.inv(Psi) @ X).real
    S_abs = np.abs(S)
    threshold_S = np.percentile(S_abs, 100 - percent) if percent > 0 else np.inf
    S_sparse_mask = (S_abs >= threshold_S).astype(int)

    display_size = 256
    vector_width = 36
    block_gap = 120
    base_x = 0

    fig = go.Figure()

    # X vector
    X_disp = resize(X.reshape(n,1), (display_size,1), anti_aliasing=False)
    fig.add_trace(go.Heatmap(
        z=X_disp.repeat(vector_width, axis=1),
        colorscale="Gray", showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size))
    ))
    base_x += vector_width + block_gap

    # Psi matrix
    Psi_disp = resize(Psi.real, (display_size, display_size), anti_aliasing=False)
    fig.add_trace(go.Heatmap(
        z=Psi_disp, colorscale="RdBu", zmid=0, showscale=False,
        x=list(range(base_x, base_x + display_size)),
        y=list(range(display_size))
    ))
    base_x += display_size + block_gap

    # S vector
    S_disp = resize(S_sparse_mask.reshape(n,1), (display_size,1),
                    anti_aliasing=False, order=0)
    fig.add_trace(go.Heatmap(
        z=S_disp.repeat(vector_width, axis=1),
        colorscale=[[0, "black"], [1, "orange"]],
        zmin=0, zmax=1, showscale=False,
        x=list(range(base_x, base_x + vector_width)),
        y=list(range(display_size))
    ))

    fig.update_layout(
        font=dict(family="Times New Roman"),
        xaxis=dict(showticklabels=False, showgrid=False, zeroline=False),
        yaxis=dict(showticklabels=False, showgrid=False, zeroline=False, scaleanchor="x"),
        plot_bgcolor="white",
        margin=dict(l=0, r=0, t=0, b=0),  # tight, no whitespace
        height=600,
        width=1200
    )

    pio.write_image(fig, filename, format="pdf")
    print(f"✅ Saved bottom row to {filename}")


In [ ]:
save_top_row(percent=20, n_side=46, filename="CStop20per.pdf")
save_bottom_row(percent=20, n_side=46, filename="CSbottom20per.pdf")
